In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, log_loss, make_scorer
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA, NMF
from sklearn.model_selection import learning_curve
from sklearn.manifold import TSNE
from umap import UMAP
from scipy.cluster.hierarchy import dendrogram, ward
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

sns.set_theme(style = 'white', palette = 'viridis')
pal = sns.color_palette('viridis')
pd.set_option('display.max_rows', 100)

## How to define the different states of the data

In [2]:
#Variable for the imported training data after cleaning up - "train"
#Variable for the feature data set - "train_X"
#Variable for the target series- "train_y"
#Variable for the  target colum - "target"
#Variable for the imported test data set - "test"
#Variable for the transformed test data set - "test_trans"
#Variable for the training data set in the cross validation loop - "X_train"
#Variable for the validation data set in the cross validation loop - "X_val"



### Downloading data and getting into a feature matrix and labels vector

In [3]:
train = pd.read_csv(r'../input/playground-series-s3e13/train.csv')
test_1 = pd.read_csv(r'../input/playground-series-s3e13/test.csv')
orig_train = pd.read_csv(r'../input/vector-borne-disease-prediction/trainn.csv')

train.drop('id', axis = 1, inplace = True)
test = test_1.drop('id', axis = 1)

target = 'prognosis'

In [4]:
train = pd.concat([train, orig_train])
print(f'There are {train.duplicated(subset = list(train)[0:-1]).value_counts()[0]} non-duplicate values out of {train.count()[0]} rows in original train dataset')

There are 959 non-duplicate values out of 959 rows in original train dataset


In [5]:
train.shape

(959, 65)

In [6]:
train_X = train.drop(target, axis = 1).copy()
train_y = train[target]

#Fix the issues that the original data set used "-"to separate the words
train_y = [prognosis.replace(' ', '_') for prognosis in train_y]
train_y = np.array(train_y)

In [7]:
train_X.shape, train_y.shape

((959, 64), (959,))

### Cross-validation strategy

In [57]:
#Cross validation strategy
seed = 42
splits = 5
#cv = RepeatedStratifiedKFold(n_splits = splits, n_repeats = 5, random_state = seed)
cv = StratifiedKFold(n_splits = splits, random_state = seed, shuffle = True)

np.random.seed(seed)

# Evaluation metric

In [9]:
def apk(actual, predicted, k=10):
    
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [10]:
def cross_val_pipe(model, train_X = train_X, train_y = train_y, target ='prognosis', cv = cv, label = ''):
    
    
    #creating encoder and transforming prognosis
    enc = LabelEncoder()
    train_y = enc.fit_transform(train_y)
    
    #initiate prediction arrays and score lists
    val_predictions = np.zeros((len(train_X), 11)) # Validation predictions are stored in a matrix with length of the number of trainijng samples and # of preds
    train_predictions = np.zeros((len(train_X), 11)) # Same for the train predictions
    train_logloss, val_logloss = [], [] #Store the results from the log_loss calc in a list
    train_map3, val_map3 = [], [] #Store the results from the log_loss calc in a list
    
    #training model, predicting prognosis probability, and evaluating log loss
    for fold, (train_idx, val_idx) in enumerate(cv.split(train_X, train_y)):
                
        model.fit(train_X.iloc[train_idx], train_y[train_idx])
        
        train_preds = model.predict_proba(train_X.iloc[train_idx])
        val_preds = model.predict_proba(train_X.iloc[val_idx])
                  
        train_predictions[train_idx] += train_preds
        val_predictions[val_idx] += val_preds
        
        train_score = log_loss(train_y[train_idx], train_preds)
        val_score = log_loss(train_y[val_idx], val_preds)
        
        train_logloss.append(train_score)
        val_logloss.append(val_score)
        
        #select three most probable prognosis based on train dataset prediction
        train_index = np.argsort(-train_preds)[:,:3] #return index of three most probable prognosis
        
        #select three most probable prognosis based on validation dataset prediction
        val_index = np.argsort(-val_preds)[:,:3]
    
        #calculate map@3
        train_score = mapk(train_y[train_idx].reshape(-1, 1), train_index, 3)
        val_score = mapk(train_y[val_idx].reshape(-1, 1), val_index, 3)
        print(f" The val_score for {fold} is {val_score}")
        
        train_map3.append(train_score)
        val_map3.append(val_score)
    
    print(f'Val log_loss   : {np.mean(val_logloss):.5f} | Train log_loss   : {np.mean(train_logloss):.5f} | {label}')
    print(f'Val MAP@3 Score: {np.mean(val_map3):.5f} | Train MAP@3 Score: {np.mean(train_map3):.5f} | {label}\n')
    
    return val_logloss, val_map3

# Models and Parameters for Multi-class Classification

In [72]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

In [11]:
#Classification models

#List of tuples
models = [
    ('log', LogisticRegression(random_state = seed, max_iter = 10000)),
    #('svc', SVC(random_state = seed, probability = True)),
    #('lda', LinearDiscriminantAnalysis()),
    #('qda', QuadraticDiscriminantAnalysis()),
    #('gauss', GaussianProcessClassifier(random_state = seed)),
    #('et', ExtraTreesClassifier(random_state = seed)),
    #('rf', RandomForestClassifier(random_state = seed)),
    #('xgb', XGBClassifier(random_state = seed, objective = 'multi:softprob', eval_metric = 'map@3')),
    ('lgb', LGBMClassifier(random_state = seed, objective = 'softmax', metric = 'softmax')),
    #('dart', LGBMClassifier(random_state = seed, objective = 'softmax', metric = 'softmax', boosting_type = 'dart')),
    #('cb', CatBoostClassifier(random_state = seed, objective = 'MultiClass', verbose = 0)),
    #('gb', GradientBoostingClassifier(random_state = seed)),
    #('hgb', HistGradientBoostingClassifier(random_state = seed)),
    #('ada', AdaBoostClassifier(random_state = seed)),
    #('knn', KNeighborsClassifier())
]

# Feature Engineering

### Dimension reduction - TNSE, PCA and other clustering based algorythms

In [12]:
class Decomp:
    def __init__(self, n_components, method="pca", scaler_method='standard'):
        self.n_components = n_components
        self.method = method
        self.scaler_method = scaler_method
        
    def dimension_reduction(self, df):
            
        X_reduced = self.dimension_method(df)
        df_comp = pd.DataFrame(X_reduced, columns=[f'{self.method.upper()}_{_}' for _ in range(self.n_components)], index=df.index)
        
        return df_comp
    
    def dimension_method(self, df):
        X = self.scaler(df)
        if self.method == "pca":
            comp = PCA(n_components=self.n_components, random_state=0)
            X_reduced = comp.fit_transform(X)
        elif self.method == "nmf":
            comp = NMF(n_components=self.n_components, random_state=0)
            X_reduced = comp.fit_transform(X)
        elif self.method == "umap":
            comp = UMAP(n_components=self.n_components, random_state=0)
            X_reduced = comp.fit_transform(X)
        elif self.method == "tsne":
            comp = TSNE(n_components=self.n_components, random_state=0) # Recommend n_components=2
            X_reduced = comp.fit_transform(X)
        else:
            raise ValueError(f"Invalid method name: {method}")
        
        self.comp = comp
        return X_reduced
    
    def scaler(self, df):
        
        _df = df.copy()
            
        if self.scaler_method == "standard":
            return StandardScaler().fit_transform(_df)
        elif self.scaler_method == "minmax":
            return MinMaxScaler().fit_transform(_df)
        elif self.scaler_method == None:
            return _df.values
        else:
            raise ValueError(f"Invalid scaler_method name")
        
    def get_columns(self):
        return [f'{self.method.upper()}_{_}' for _ in range(self.n_components)]
    
    def transform(self, df):
        X = self.scaler(df)
        X_reduced = self.comp.transform(X)
        df_comp = pd.DataFrame(X_reduced, columns=[f'{self.method.upper()}_{_}' for _ in range(self.n_components)], index=df.index)
        
        return df_comp
    @property
        
    def get_explained_variance_ratio(self):
        
        return np.sum(self.comp.explained_variance_ratio_)

In [13]:
def decomp_concat(df):
    global method
    decomp = Decomp(n_components=3, method=method, scaler_method=None)
    df_1 = decomp.dimension_reduction(df).reset_index(drop = True)
    print(f" the shape of df_1 is: {df_1.shape}")
    #df = df.reset_index(inplace = True)
    df = pd.merge(df, df_1, left_index=True, right_index=True)
    print(f" the shape of df_2 2 is: {df.shape}")
    #df = pd.concat([train_X, df], axis=1)
    #df = df.reset_index(inplace = True)
    return df

In [17]:
def decomp_concat(df):
    global method
    decomp = Decomp(n_components=9, method=method, scaler_method=None)
    df_1 = decomp.dimension_reduction(df).reset_index(drop = True)
    print(f" the shape of df_1 is: {df_1.shape}")
    #df = df.reset_index(inplace = True)
    df = pd.merge(df, df_1, left_index=True, right_index=True)
    print(f" the shape of df_2 2 is: {df.shape}")
    #df = pd.concat([train_X, df], axis=1)
    #df = df.reset_index(inplace = True)
    return df

### Pipeline

In [25]:
# Function that is sometimes needed especially with concat data
def reset_index(dataframe):
    dataframe = dataframe.reset_index(inplace = False)
    return dataframe

get_reset_index = FunctionTransformer(reset_index, validate=False)

In [18]:
pca_func_trans = FunctionTransformer(decomp_concat)

In [19]:
pca_func_trans

FunctionTransformer(func=<function decomp_concat at 0x72ab3c8ebef0>)

In [21]:
#Set up a logistic regression pipeline
#No feature engineering part )
#pca_pipe = make_pipeline(PCA(n_components=2, random_state=0), LogisticRegression(random_state = seed, max_iter = 10000))
pca_pipe = make_pipeline( get_reset_index, pca_func_trans, LogisticRegression(random_state = seed, max_iter = 10000))
umap_pipe = make_pipeline(UMAP(n_components=3, random_state=0), LogisticRegression(random_state = seed, max_iter = 10000))

In [22]:
pca_pipe

Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function reset_index at 0x72ab3cbd0320>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function decomp_concat at 0x72ab3c8ebef0>)),
                ('logisticregression',
                 LogisticRegression(max_iter=10000, random_state=42))])

In [23]:
cross_val_pipe(pca_pipe)

 the shape of df_1 is: (639, 9)
 the shape of df_2 2 is: (639, 74)
 the shape of df_1 is: (639, 9)
 the shape of df_2 2 is: (639, 74)
 the shape of df_1 is: (320, 9)
 the shape of df_2 2 is: (320, 74)
 The val_score for 0 is 0.3234375
 the shape of df_1 is: (639, 9)
 the shape of df_2 2 is: (639, 74)
 the shape of df_1 is: (639, 9)
 the shape of df_2 2 is: (639, 74)
 the shape of df_1 is: (320, 9)
 the shape of df_2 2 is: (320, 74)
 The val_score for 1 is 0.29166666666666663
 the shape of df_1 is: (640, 9)
 the shape of df_2 2 is: (640, 74)
 the shape of df_1 is: (640, 9)
 the shape of df_2 2 is: (640, 74)
 the shape of df_1 is: (319, 9)
 the shape of df_2 2 is: (319, 74)
 The val_score for 2 is 0.2507836990595611
Val log_loss   : 3.67350 | Train log_loss   : 1.00557 | 
Val MAP@3 Score: 0.28863 | Train MAP@3 Score: 0.68039 | 



([2.9812251379993095, 3.914486132290827, 4.124775095667349],
 [0.3234375, 0.29166666666666663, 0.2507836990595611])

# Create pipeline where the best estimator type can selected

In [ ]:
pca_pipe = make_pipeline(PCA(n_components=2, random_state=0), model)
#pca_pipe = make_pipeline(PCA(n_components=2, random_state=0), LogisticRegression(random_state = seed, max_iter = 10000))
#pca_pipe = make_pipeline( get_reset_index, pca_func_trans, LogisticRegression(random_state = seed, max_iter = 10000))
#umap_pipe = make_pipeline(UMAP(n_components=3, random_state=0), LogisticRegression(random_state = seed, max_iter = 10000))

### Create the cross-validation pipeline that stores the predictions

### Loop for fitting the models and making predictions

In [32]:

def cross_val_pipe(model, train_X = train_X, train_y = train_y, target ='prognosis', cv = cv, label = ''):
    
    
    #creating encoder and transforming prognosis
    enc = LabelEncoder()
    train_y = enc.fit_transform(train_y)
    
    #initiate prediction arrays and score lists
    val_predictions = np.zeros((len(train_X), 11)) # Validation predictions are stored in a matrix with length of the number of trainijng samples and # of preds
    train_predictions = np.zeros((len(train_X), 11)) # Same for the train predictions
    train_logloss, val_logloss = [], [] #Store the results from the log_loss calc in a list
    train_map3, val_map3 = [], [] #Store the results from the log_loss calc in a list
    
    #training model, predicting prognosis probability, and evaluating log loss
    for fold, (train_idx, val_idx) in enumerate(cv.split(train_X, train_y)):
                
        model.fit(train_X.iloc[train_idx], train_y[train_idx])
        
        train_preds = model.predict_proba(train_X.iloc[train_idx])
        val_preds = model.predict_proba(train_X.iloc[val_idx])
                  
        train_predictions[train_idx] += train_preds
        val_predictions[val_idx] += val_preds
        
        train_score = log_loss(train_y[train_idx], train_preds)
        val_score = log_loss(train_y[val_idx], val_preds)
        
        train_logloss.append(train_score)
        val_logloss.append(val_score)
        
        #select three most probable prognosis based on train dataset prediction
        train_index = np.argsort(-train_preds)[:,:3] #return index of three most probable prognosis
        
        #select three most probable prognosis based on validation dataset prediction
        val_index = np.argsort(-val_preds)[:,:3]
    
        #calculate map@3
        train_score = mapk(train_y[train_idx].reshape(-1, 1), train_index, 3)
        val_score = mapk(train_y[val_idx].reshape(-1, 1), val_index, 3)
        print(f" The val_score for {fold} is {val_score}")
        
        train_map3.append(train_score)
        val_map3.append(val_score)
        val_predictions.append(val_predictions)
    
    print(f'Val log_loss   : {np.mean(val_logloss):.5f} | Train log_loss   : {np.mean(train_logloss):.5f} | {label}')
    print(f'Val MAP@3 Score: {np.mean(val_map3):.5f} | Train MAP@3 Score: {np.mean(train_map3):.5f} | {label}\n')
    
    return val_logloss, val_map3, val_predictions

### Class-basd approach

In [47]:
# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)

In [51]:
rf = RandomForestClassifier(random_state = 49, **rf_params)
et = ExtraTreesClassifier(random_state = 49, **et_params)
gb = ExtraTreesClassifier(random_state = 49, **gb_params)

In [ ]:
#Cross-validation applications
# Model selection
# Determining the error metric for the pipeline
# Predicting the labels for the training data sets - Only use them from cross-validation

In [74]:
enc = LabelEncoder()
train_y = enc.fit_transform(train_y)

In [88]:
val_predictions = np.zeros((len(train_X), 11))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [81]:
model.fit(train_X.iloc[1:200], train_y[1:200])

LogisticRegression(max_iter=10000, random_state=42)

In [83]:
val_preds = model.predict_proba(train_X.iloc[201:300])

In [87]:
val_preds[1:5].shape

(4, 11)

In [91]:
val_predictions[1:10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
def cross_val_pred(model, train_X = train_X, train_y = train_y, target ='prognosis', cv = cv, label = ''):
    
    
    #initiate prediction arrays and score lists
    val_predictions = np.zeros((len(train_X), 3)) # Validation predictions are stored in a matrix with length of the number of trainijng samples and # of preds
    
    #training model, predicting prognosis probability, and evaluating log loss
    for fold, (train_idx, val_idx) in enumerate(cv.split(train_X, train_y)):
                
        model.fit(train_X.iloc[train_idx], train_y[train_idx])
        
        
        val_preds = model.predict_proba(train_X.iloc[val_idx])
                  
       
        val_predictions[val_idx] = val_preds
        
        train_score = log_loss(train_y[train_idx], train_preds)
        val_score = log_loss(train_y[val_idx], val_preds)
        
        train_logloss.append(train_score)
        val_logloss.append(val_score)
        
        #select three most probable prognosis based on train dataset prediction
        train_index = np.argsort(-train_preds)[:,:3] #return index of three most probable prognosis
        
        #select three most probable prognosis based on validation dataset prediction
        val_index = np.argsort(-val_preds)[:,:3]
    
        #calculate map@3
        train_score = mapk(train_y[train_idx].reshape(-1, 1), train_index, 3)
        val_score = mapk(train_y[val_idx].reshape(-1, 1), val_index, 3)
        print(f" The val_score for {fold} is {val_score}")
        
        train_map3.append(train_score)
        val_map3.append(val_score)
    
    print(f'Val log_loss   : {np.mean(val_logloss):.5f} | Train log_loss   : {np.mean(train_logloss):.5f} | {label}')
    print(f'Val MAP@3 Score: {np.mean(val_map3):.5f} | Train MAP@3 Score: {np.mean(train_map3):.5f} | {label}\n')
    
    return val_logloss, val_map3

In [73]:
#Comment - Determine the best hyperparamter und feature engineering strategy separately

#Input - list of pipelines, the training data, the cross validation strategy
#Output dataframe with the predictions for each classifier

#Get empty df

pred_df = pd.DataFrame()

for (label, pipeline) in pipelines:
    pred_df[label] = cross_val_pipe(pipeline, label = label)

    

# Workflow

In [ ]:
# 1.) Create a function to load the data and do the general feature engineering steps
# 2.) Define the cross-validation strategy
# 3.) Define the loss function and the metric
# 4.) Set up the feature engineering pipeline for each classifier
# 5.) Perform hyperparameter optimization
# 6.) Fit, predict, score 